In [48]:
import pandas as pd
import numpy as np

In [56]:
data = pd.read_csv("rating_final.csv")
data = data[["userID", "placeID", "rating"]]

In [57]:
data.head()

,userID,placeID,rating
0,1077,135085,2
1,1077,135038,2
2,1077,132825,2
3,1077,135060,1
4,1068,135104,1


In [58]:
from sklearn.cluster import KMeans

In [51]:
# Create the utility matrix
umatrix = data.pivot(columns="userID", values="rating")

In [52]:
undefined_values = umatrix.isna().sum().sum()
total_values = umatrix.shape[0] * umatrix.shape[1]

sparsity = int(undefined_values / total_values * 10000)
print(f"{sparsity/100.0}% empty")

99.27% empty


In [53]:
# Replace NaN with 0
umatrix.fillna(0, inplace=True)

# Factorize using Numpy SVD
np.linalg.svd(umatrix, full_matrices=False)

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 array([7.21110255e+00, 7.14142843e+00, 7.00000000e+00, 7.00000000e+00,
        6.78232998e+00, 6.70820393e+00, 6.55743852e+00, 6.55743852e+00,
        6.40312424e+00, 6.40312424e+00, 6.40312424e+00, 6.40312424e+00,
        6.08276253e+00, 6.00000000e+00, 5.91607978e+00, 5.83095189e+00,
        5.83095189e+00, 5.83095189e+00, 5.65685425e+00, 5.56776436e+00,
        5.56776436e+00, 5.47722558e+00, 5.38516481e+00, 5.38516481e+00,
        5.38516481e+00, 5.29150262e+00, 5.29150262e+00, 5.29150262e+00,
        5.29150262e+00, 5.19615242e+00, 5.09901951e+00, 5.00000000e+00,
        5.00000000e+00, 5.00000000e+00, 5.00000000e+00, 4.89897949e+00,
        4.89897949e+00, 4.89897949e+00, 4.89897949e+00, 4.79583152e+00,
        4.79583152e+00, 4.79583

    ## Hybrid recommender
![alt text](feature_augment_graph.jpeg "Feature augment")
Source: https://medium.com/analytics-vidhya/7-types-of-hybrid-recommendation-system-3e4f78266ad8